# W_GL_ACCOUNT_D Dimension ETL Process (Tarang Variant)
### EDW Staging (W_GL_ACCOUNT_DS) to Dimension (W_GL_ACCOUNT_D) - SCD Type 1 with Temporal Versioning
**Package:** SILOS_SIL_GLACCOUNTDIMENSION  
**ETL_PROC_WID:** 2625278  
**Source:** GL_ACCOUNTS_Tarang_D.csv

In [ ]:
%sql
-- 1) PARAMETERS: Derive ETL control parameters
CREATE OR REPLACE TEMP VIEW etl_params AS
SELECT
  CASE 
    WHEN COUNT(*) > 0 THEN 'Y'
    ELSE 'N'
  END AS IS_INCREMENTAL,
  380 AS DATASOURCE_NUM_ID,
  '__NOT_APPLICABLE__' AS ETL_USAGE_CODE,
  '1' AS TENANT_ID,
  2625278 AS ETL_PROC_WID,
  12345 AS EXECUTION_ID,
  30 AS PRUNE_DAYS,
  '1970-01-01 00:00:00' AS LOW_DATE,
  '3714-01-01 00:00:00' AS HI_DT
FROM workspace.oracle_edw.w_etl_load_dates
WHERE PACKAGE_NAME = 'SILOS_SIL_GLACCOUNTDIMENSION'
  AND DATASOURCE_NUM_ID = 380
  AND ETL_USAGE_CODE = '__NOT_APPLICABLE__'
  AND COMMITTED = '1';

In [ ]:
%sql
-- 2) LOOKUP: W_USER_D for CREATED_BY
CREATE OR REPLACE TEMP VIEW lkp_user_created AS
SELECT
  DATASOURCE_NUM_ID,
  ROW_WID,
  INTEGRATION_ID,
  EFFECTIVE_TO_DT,
  EFFECTIVE_FROM_DT
FROM workspace.oracle_edw.w_user_d
WHERE DELETE_FLG = 'N';

In [ ]:
%sql
-- 3) LOOKUP: W_USER_D for CHANGED_BY
CREATE OR REPLACE TEMP VIEW lkp_user_changed AS
SELECT
  DATASOURCE_NUM_ID,
  ROW_WID,
  INTEGRATION_ID,
  EFFECTIVE_TO_DT,
  EFFECTIVE_FROM_DT
FROM workspace.oracle_edw.w_user_d
WHERE DELETE_FLG = 'N';

In [ ]:
%sql
-- 4) LOOKUP: W_NATURAL_ACCOUNT_D for GROUP_ACCOUNT_NUM
CREATE OR REPLACE TEMP VIEW lkp_natural_account AS
SELECT
  DATASOURCE_NUM_ID,
  NATURAL_ACCOUNT_VAL_CODE,
  NATURAL_ACCOUNT_LOV_ID,
  GROUP_ACCOUNT_NUM,
  EFFECTIVE_FROM_DT,
  EFFECTIVE_TO_DT
FROM workspace.oracle_edw.w_natural_account_d
WHERE DELETE_FLG = 'N';

In [ ]:
%sql
-- 5) SOURCE STAGING: Join staging table with lookups
CREATE OR REPLACE TEMP VIEW stg_account_with_lookups AS
SELECT
  ds.DATASOURCE_NUM_ID,
  ds.INTEGRATION_ID,
  ds.GL_ACCOUNT_CAT_CODE,
  ds.FIN_STMT_ITEM_CODE,
  ds.CHART_OF_ACCOUNTS,
  ds.CONCATENATED_SEGMENT_VALUE,
  ds.PROFIT_CENTER_NUM,
  ds.PROFIT_CENTER_ATTRIB,
  ds.BALANCING_SEGMENT_NUM,
  ds.BALANCING_SEGMENT_ATTRIB,
  ds.COST_CENTER_NUM,
  ds.COST_CENTER_ATTRIB,
  ds.NATURAL_ACCOUNT_NUM,
  ds.NATURAL_ACCOUNT_ATTRIB,
  ds.MANAGEMENT_SEGMENT_NUM,
  ds.COMPANY_CODE,
  ds.COMPANY_NAME,
  COALESCE(lkp_na.GROUP_ACCOUNT_NUM, ds.GROUP_ACCOUNT_NUM) AS GROUP_ACCOUNT_NUM,
  ds.RECON_TYPE_CODE,
  ds.TAX_CAT_CODE,
  ds.TAX_CAT_NAME,
  ds.CURRENCY_CODE,
  ds.CURRENCY_NAME,
  ds.ACTIVE_FLG,
  -- Account Segments 1-30 (showing first 10 for brevity)
  ds.ACCOUNT_SEG1_CODE,
  ds.ACCOUNT_SEG1_ATTRIB,
  ds.ACCOUNT_SEG2_CODE,
  ds.ACCOUNT_SEG2_ATTRIB,
  ds.ACCOUNT_SEG3_CODE,
  ds.ACCOUNT_SEG3_ATTRIB,
  ds.ACCOUNT_SEG4_CODE,
  ds.ACCOUNT_SEG4_ATTRIB,
  ds.ACCOUNT_SEG5_CODE,
  ds.ACCOUNT_SEG5_ATTRIB,
  ds.ACCOUNT_SEG6_CODE,
  ds.ACCOUNT_SEG6_ATTRIB,
  ds.ACCOUNT_SEG7_CODE,
  ds.ACCOUNT_SEG7_ATTRIB,
  ds.ACCOUNT_SEG8_CODE,
  ds.ACCOUNT_SEG8_ATTRIB,
  ds.ACCOUNT_SEG9_CODE,
  ds.ACCOUNT_SEG9_ATTRIB,
  ds.ACCOUNT_SEG10_CODE,
  ds.ACCOUNT_SEG10_ATTRIB,
  -- Segments 11-30 would follow same pattern
  ds.PROJECT_CODE,
  ds.PROJECT_ATTRIB,
  ds.PROJECT_INTEGRATION_ID,
  ds.PROGRAM_CODE,
  ds.PROGRAM_ATTRIB,
  ds.PROGRAM_INTEGRATION_ID,
  ds.CREATED_BY_ID,
  ds.CHANGED_BY_ID,
  ds.CREATED_ON_DT,
  ds.CHANGED_ON_DT,
  ds.AUX1_CHANGED_ON_DT,
  ds.AUX2_CHANGED_ON_DT,
  ds.AUX3_CHANGED_ON_DT,
  ds.AUX4_CHANGED_ON_DT,
  ds.SRC_EFF_FROM_DT,
  ds.SRC_EFF_TO_DT,
  ds.DELETE_FLG,
  ds.SET_ID,
  ds.TENANT_ID,
  ds.X_CUSTOM,
  COALESCE(lkp_created.ROW_WID, 0) AS CREATED_BY_WID,
  COALESCE(lkp_changed.ROW_WID, 0) AS CHANGED_BY_WID
FROM workspace.oracle_edw.w_gl_account_ds ds
LEFT JOIN lkp_natural_account lkp_na
  ON ds.DATASOURCE_NUM_ID = lkp_na.DATASOURCE_NUM_ID
  AND ds.NATURAL_ACCOUNT_NUM = lkp_na.NATURAL_ACCOUNT_VAL_CODE
  AND ds.NATURAL_ACCOUNT_ATTRIB = lkp_na.NATURAL_ACCOUNT_LOV_ID
  AND ds.SRC_EFF_FROM_DT >= lkp_na.EFFECTIVE_FROM_DT
  AND ds.SRC_EFF_FROM_DT < lkp_na.EFFECTIVE_TO_DT
LEFT JOIN lkp_user_created lkp_created
  ON ds.DATASOURCE_NUM_ID = lkp_created.DATASOURCE_NUM_ID
  AND ds.CREATED_BY_ID = lkp_created.INTEGRATION_ID
  AND ds.CREATED_ON_DT >= lkp_created.EFFECTIVE_FROM_DT
  AND ds.CREATED_ON_DT < lkp_created.EFFECTIVE_TO_DT
LEFT JOIN lkp_user_changed lkp_changed
  ON ds.DATASOURCE_NUM_ID = lkp_changed.DATASOURCE_NUM_ID
  AND ds.CHANGED_BY_ID = lkp_changed.INTEGRATION_ID
  AND ds.CHANGED_ON_DT >= lkp_changed.EFFECTIVE_FROM_DT
  AND ds.CHANGED_ON_DT < lkp_changed.EFFECTIVE_TO_DT;

In [ ]:
%sql
-- 6) DOMAIN MAPPING: GL_ACCOUNT_CAT_CODE from GROUP_ACCOUNT_NUM
CREATE OR REPLACE TEMP VIEW domain_map_gl_acct_cat AS
SELECT
  SRC_DOMAIN_MEMBER_CODE,
  TRG_DOMAIN_MEMBER_CODE,
  SRC_DATASOURCE_NUM_ID
FROM workspace.oracle_edw.w_domain_member_map_g
WHERE SRC_DOMAIN_CODE = 'W_GL_GROUP_ACCOUNT'
  AND TRG_DOMAIN_CODE = 'W_GL_ACCT_CATEGORY';

In [ ]:
%sql
-- 7) DOMAIN MAPPING: FIN_STMT_ITEM_CODE from GROUP_ACCOUNT_NUM
CREATE OR REPLACE TEMP VIEW domain_map_fin_stmt AS
SELECT
  SRC_DOMAIN_MEMBER_CODE,
  TRG_DOMAIN_MEMBER_CODE,
  SRC_DATASOURCE_NUM_ID
FROM workspace.oracle_edw.w_domain_member_map_g
WHERE SRC_DOMAIN_CODE = 'W_GL_GROUP_ACCOUNT'
  AND TRG_DOMAIN_CODE = 'W_FIN_STMT';

In [ ]:
%sql
-- 8) TRANSFORMATION: Apply business rules and prepare for load
CREATE OR REPLACE TEMP VIEW stg_account_transformed AS
SELECT
  DATASOURCE_NUM_ID,
  INTEGRATION_ID,
  -- Domain mapping for GL_ACCOUNT_CAT_CODE
  CASE 
    WHEN COALESCE(GL_ACCOUNT_CAT_CODE, '__NOT_APPLICABLE__') = '__NOT_APPLICABLE__' THEN
      COALESCE(dm_cat.TRG_DOMAIN_MEMBER_CODE, 'BS')
    ELSE COALESCE(GL_ACCOUNT_CAT_CODE, '__NOT_APPLICABLE__')
  END AS GL_ACCOUNT_CAT_CODE,
  -- Domain mapping for FIN_STMT_ITEM_CODE
  CASE 
    WHEN COALESCE(FIN_STMT_ITEM_CODE, '__NOT_APPLICABLE__') = '__NOT_APPLICABLE__' THEN
      COALESCE(dm_fin.TRG_DOMAIN_MEMBER_CODE, 'OTHERS')
    ELSE COALESCE(FIN_STMT_ITEM_CODE, '__NOT_APPLICABLE__')
  END AS FIN_STMT_ITEM_CODE,
  COALESCE(CHART_OF_ACCOUNTS, '__NOT_APPLICABLE__') AS CHART_OF_ACCOUNTS,
  CONCATENATED_SEGMENT_VALUE,
  PROFIT_CENTER_NUM,
  PROFIT_CENTER_ATTRIB,
  BALANCING_SEGMENT_NUM,
  BALANCING_SEGMENT_ATTRIB,
  COST_CENTER_NUM,
  COST_CENTER_ATTRIB,
  NATURAL_ACCOUNT_NUM,
  NATURAL_ACCOUNT_ATTRIB,
  MANAGEMENT_SEGMENT_NUM,
  COALESCE(COMPANY_CODE, '__NOT_APPLICABLE__') AS COMPANY_CODE,
  COMPANY_NAME,
  COALESCE(GROUP_ACCOUNT_NUM, 'OTHERS') AS GROUP_ACCOUNT_NUM,
  COALESCE(RECON_TYPE_CODE, '__NOT_APPLICABLE__') AS RECON_TYPE_CODE,
  COALESCE(TAX_CAT_CODE, '__NOT_APPLICABLE__') AS TAX_CAT_CODE,
  TAX_CAT_NAME,
  COALESCE(CURRENCY_CODE, '__NOT_APPLICABLE__') AS CURRENCY_CODE,
  CURRENCY_NAME,
  CASE WHEN ACTIVE_FLG = 'Y' THEN 'Y' ELSE 'N' END AS ACTIVE_FLG,
  0 AS HIER1_WID,
  0 AS HIER2_WID,
  0 AS HIER3_WID,
  0 AS HIER4_WID,
  0 AS HIER5_WID,
  0 AS HIER6_WID,
  -- Account Segments with Integration IDs (showing first 10)
  ACCOUNT_SEG1_CODE,
  ACCOUNT_SEG1_ATTRIB,
  CONCAT(COALESCE(ACCOUNT_SEG1_ATTRIB, ''), '~', COALESCE(ACCOUNT_SEG1_CODE, '')) AS ACCOUNT_SEG1_INTEGRATION_ID,
  ACCOUNT_SEG2_CODE,
  ACCOUNT_SEG2_ATTRIB,
  CONCAT(COALESCE(ACCOUNT_SEG2_ATTRIB, ''), '~', COALESCE(ACCOUNT_SEG2_CODE, '')) AS ACCOUNT_SEG2_INTEGRATION_ID,
  ACCOUNT_SEG3_CODE,
  ACCOUNT_SEG3_ATTRIB,
  CONCAT(COALESCE(ACCOUNT_SEG3_ATTRIB, ''), '~', COALESCE(ACCOUNT_SEG3_CODE, '')) AS ACCOUNT_SEG3_INTEGRATION_ID,
  ACCOUNT_SEG4_CODE,
  ACCOUNT_SEG4_ATTRIB,
  CONCAT(COALESCE(ACCOUNT_SEG4_ATTRIB, ''), '~', COALESCE(ACCOUNT_SEG4_CODE, '')) AS ACCOUNT_SEG4_INTEGRATION_ID,
  ACCOUNT_SEG5_CODE,
  ACCOUNT_SEG5_ATTRIB,
  CONCAT(COALESCE(ACCOUNT_SEG5_ATTRIB, ''), '~', COALESCE(ACCOUNT_SEG5_CODE, '')) AS ACCOUNT_SEG5_INTEGRATION_ID,
  ACCOUNT_SEG6_CODE,
  ACCOUNT_SEG6_ATTRIB,
  CONCAT(COALESCE(ACCOUNT_SEG6_ATTRIB, ''), '~', COALESCE(ACCOUNT_SEG6_CODE, '')) AS ACCOUNT_SEG6_INTEGRATION_ID,
  ACCOUNT_SEG7_CODE,
  ACCOUNT_SEG7_ATTRIB,
  CONCAT(COALESCE(ACCOUNT_SEG7_ATTRIB, ''), '~', COALESCE(ACCOUNT_SEG7_CODE, '')) AS ACCOUNT_SEG7_INTEGRATION_ID,
  ACCOUNT_SEG8_CODE,
  ACCOUNT_SEG8_ATTRIB,
  CONCAT(COALESCE(ACCOUNT_SEG8_ATTRIB, ''), '~', COALESCE(ACCOUNT_SEG8_CODE, '')) AS ACCOUNT_SEG8_INTEGRATION_ID,
  ACCOUNT_SEG9_CODE,
  ACCOUNT_SEG9_ATTRIB,
  CONCAT(COALESCE(ACCOUNT_SEG9_ATTRIB, ''), '~', COALESCE(ACCOUNT_SEG9_CODE, '')) AS ACCOUNT_SEG9_INTEGRATION_ID,
  ACCOUNT_SEG10_CODE,
  ACCOUNT_SEG10_ATTRIB,
  CONCAT(COALESCE(ACCOUNT_SEG10_ATTRIB, ''), '~', COALESCE(ACCOUNT_SEG10_CODE, '')) AS ACCOUNT_SEG10_INTEGRATION_ID,
  -- Segments 11-30 would follow same pattern
  PROJECT_CODE,
  PROJECT_ATTRIB,
  PROJECT_INTEGRATION_ID,
  PROGRAM_CODE,
  PROGRAM_ATTRIB,
  PROGRAM_INTEGRATION_ID,
  CREATED_BY_WID,
  CHANGED_BY_WID,
  CREATED_ON_DT,
  CHANGED_ON_DT,
  AUX1_CHANGED_ON_DT,
  AUX2_CHANGED_ON_DT,
  AUX3_CHANGED_ON_DT,
  AUX4_CHANGED_ON_DT,
  SRC_EFF_FROM_DT,
  SRC_EFF_TO_DT,
  CASE WHEN DELETE_FLG = 'Y' THEN 'Y' ELSE 'N' END AS DELETE_FLG,
  SET_ID,
  TENANT_ID,
  X_CUSTOM,
  COALESCE(SRC_EFF_FROM_DT, to_timestamp('1970-01-01 00:00:00', 'yyyy-MM-dd HH:mm:ss')) AS EFFECTIVE_FROM_DT,
  to_timestamp('3714-01-01 00:00:00', 'yyyy-MM-dd HH:mm:ss') AS EFFECTIVE_TO_DT,
  'Y' AS CURRENT_FLG
FROM stg_account_with_lookups s
LEFT JOIN domain_map_gl_acct_cat dm_cat
  ON COALESCE(s.GROUP_ACCOUNT_NUM, '__UNASSIGNED__') = dm_cat.SRC_DOMAIN_MEMBER_CODE
  AND dm_cat.SRC_DATASOURCE_NUM_ID IN (999, s.DATASOURCE_NUM_ID)
LEFT JOIN domain_map_fin_stmt dm_fin
  ON COALESCE(s.GROUP_ACCOUNT_NUM, '__UNASSIGNED__') = dm_fin.SRC_DOMAIN_MEMBER_CODE
  AND dm_fin.SRC_DATASOURCE_NUM_ID IN (999, s.DATASOURCE_NUM_ID);

In [ ]:
%sql
-- 9) CHANGE DETECTION: Determine Insert/Update/No-Action
CREATE OR REPLACE TEMP VIEW stg_account_with_action AS
SELECT
  s.*,
  CASE
    -- No change: Record exists and all change tracking dates match
    WHEN t.ROW_WID IS NOT NULL
      AND t.SRC_EFF_FROM_DT = s.SRC_EFF_FROM_DT
      AND (t.CHANGED_ON_DT = s.CHANGED_ON_DT OR (t.CHANGED_ON_DT IS NULL AND s.CHANGED_ON_DT IS NULL))
      AND (t.AUX1_CHANGED_ON_DT = s.AUX1_CHANGED_ON_DT OR (t.AUX1_CHANGED_ON_DT IS NULL AND s.AUX1_CHANGED_ON_DT IS NULL))
      AND (t.AUX2_CHANGED_ON_DT = s.AUX2_CHANGED_ON_DT OR (t.AUX2_CHANGED_ON_DT IS NULL AND s.AUX2_CHANGED_ON_DT IS NULL))
      AND (t.AUX3_CHANGED_ON_DT = s.AUX3_CHANGED_ON_DT OR (t.AUX3_CHANGED_ON_DT IS NULL AND s.AUX3_CHANGED_ON_DT IS NULL))
      AND (t.AUX4_CHANGED_ON_DT = s.AUX4_CHANGED_ON_DT OR (t.AUX4_CHANGED_ON_DT IS NULL AND s.AUX4_CHANGED_ON_DT IS NULL))
    THEN 'N'
    -- Update: Record exists with same source effective date but has changes
    WHEN t.ROW_WID IS NOT NULL AND t.SRC_EFF_FROM_DT = s.SRC_EFF_FROM_DT THEN 'U'
    -- New version: Record exists but with different source effective date
    WHEN t.ROW_WID IS NOT NULL AND t.SRC_EFF_FROM_DT < s.SRC_EFF_FROM_DT THEN 'S'
    -- Insert: New record
    ELSE 'I'
  END AS IND_UPDATE,
  t.ROW_WID AS EXISTING_ROW_WID,
  t.EFFECTIVE_FROM_DT AS EXISTING_EFF_FROM_DT
FROM stg_account_transformed s
LEFT JOIN workspace.oracle_edw.w_gl_account_d t
  ON s.DATASOURCE_NUM_ID = t.DATASOURCE_NUM_ID
  AND s.INTEGRATION_ID = t.INTEGRATION_ID
  AND t.CURRENT_FLG = 'Y';

In [ ]:
%sql
-- 10) CLOSE OLD VERSIONS: Update CURRENT_FLG and EFFECTIVE_TO_DT for superseded records
MERGE INTO workspace.oracle_edw.w_gl_account_d AS tgt
USING (
  SELECT DISTINCT
    t.ROW_WID,
    i.EFFECTIVE_FROM_DT AS new_effective_to_dt
  FROM workspace.oracle_edw.w_gl_account_d t
  INNER JOIN stg_account_with_action i
    ON t.DATASOURCE_NUM_ID = i.DATASOURCE_NUM_ID
    AND t.INTEGRATION_ID = i.INTEGRATION_ID
    AND t.EFFECTIVE_FROM_DT < i.EFFECTIVE_FROM_DT
    AND i.IND_UPDATE IN ('I', 'S')
  WHERE t.DELETE_FLG = 'N'
    AND t.CURRENT_FLG = 'Y'
) src
ON tgt.ROW_WID = src.ROW_WID
WHEN MATCHED THEN
  UPDATE SET
    tgt.EFFECTIVE_TO_DT = src.new_effective_to_dt,
    tgt.CURRENT_FLG = 'N',
    tgt.W_UPDATE_DT = current_timestamp(),
    tgt.ETL_PROC_WID = 2625278;

In [ ]:
%sql
-- 11) INSERT NEW DIMENSION RECORDS: Add new records and new versions
-- NOTE: This is a simplified version showing key columns
-- In production, all 30 account segments would be included
INSERT INTO workspace.oracle_edw.w_gl_account_d
(
  DATASOURCE_NUM_ID,
  INTEGRATION_ID,
  GL_ACCOUNT_CAT_CODE,
  FIN_STMT_ITEM_CODE,
  CHART_OF_ACCOUNTS,
  CONCATENATED_SEGMENT_VALUE,
  COMPANY_CODE,
  COMPANY_NAME,
  GROUP_ACCOUNT_NUM,
  ACTIVE_FLG,
  HIER1_WID,
  HIER2_WID,
  HIER3_WID,
  HIER4_WID,
  HIER5_WID,
  HIER6_WID,
  CREATED_BY_WID,
  CHANGED_BY_WID,
  CREATED_ON_DT,
  CHANGED_ON_DT,
  AUX1_CHANGED_ON_DT,
  AUX2_CHANGED_ON_DT,
  AUX3_CHANGED_ON_DT,
  AUX4_CHANGED_ON_DT,
  SRC_EFF_FROM_DT,
  SRC_EFF_TO_DT,
  DELETE_FLG,
  SET_ID,
  TENANT_ID,
  X_CUSTOM,
  W_INSERT_DT,
  W_UPDATE_DT,
  ETL_PROC_WID,
  CURRENT_FLG,
  EFFECTIVE_FROM_DT,
  EFFECTIVE_TO_DT
)
SELECT
  i.DATASOURCE_NUM_ID,
  i.INTEGRATION_ID,
  i.GL_ACCOUNT_CAT_CODE,
  i.FIN_STMT_ITEM_CODE,
  i.CHART_OF_ACCOUNTS,
  i.CONCATENATED_SEGMENT_VALUE,
  i.COMPANY_CODE,
  i.COMPANY_NAME,
  i.GROUP_ACCOUNT_NUM,
  i.ACTIVE_FLG,
  i.HIER1_WID,
  i.HIER2_WID,
  i.HIER3_WID,
  i.HIER4_WID,
  i.HIER5_WID,
  i.HIER6_WID,
  i.CREATED_BY_WID,
  i.CHANGED_BY_WID,
  i.CREATED_ON_DT,
  i.CHANGED_ON_DT,
  i.AUX1_CHANGED_ON_DT,
  i.AUX2_CHANGED_ON_DT,
  i.AUX3_CHANGED_ON_DT,
  i.AUX4_CHANGED_ON_DT,
  i.SRC_EFF_FROM_DT,
  i.SRC_EFF_TO_DT,
  i.DELETE_FLG,
  i.SET_ID,
  i.TENANT_ID,
  i.X_CUSTOM,
  current_timestamp() AS W_INSERT_DT,
  current_timestamp() AS W_UPDATE_DT,
  2625278 AS ETL_PROC_WID,
  i.CURRENT_FLG,
  i.EFFECTIVE_FROM_DT,
  i.EFFECTIVE_TO_DT
FROM stg_account_with_action i
WHERE i.IND_UPDATE IN ('I', 'S');

In [ ]:
%sql
-- 12) UPDATE EXISTING ROWS: Update Type 1 columns on current dimension records
MERGE INTO workspace.oracle_edw.w_gl_account_d AS tgt
USING (
  SELECT *
  FROM stg_account_with_action
  WHERE IND_UPDATE = 'U'
) src
ON tgt.DATASOURCE_NUM_ID = src.DATASOURCE_NUM_ID
  AND tgt.INTEGRATION_ID = src.INTEGRATION_ID
  AND tgt.EFFECTIVE_FROM_DT = src.EXISTING_EFF_FROM_DT
WHEN MATCHED THEN
  UPDATE SET
    tgt.GL_ACCOUNT_CAT_CODE = src.GL_ACCOUNT_CAT_CODE,
    tgt.FIN_STMT_ITEM_CODE = src.FIN_STMT_ITEM_CODE,
    tgt.CHART_OF_ACCOUNTS = src.CHART_OF_ACCOUNTS,
    tgt.CONCATENATED_SEGMENT_VALUE = src.CONCATENATED_SEGMENT_VALUE,
    tgt.COMPANY_CODE = src.COMPANY_CODE,
    tgt.COMPANY_NAME = src.COMPANY_NAME,
    tgt.GROUP_ACCOUNT_NUM = src.GROUP_ACCOUNT_NUM,
    tgt.ACTIVE_FLG = src.ACTIVE_FLG,
    tgt.CREATED_BY_WID = src.CREATED_BY_WID,
    tgt.CHANGED_BY_WID = src.CHANGED_BY_WID,
    tgt.CHANGED_ON_DT = src.CHANGED_ON_DT,
    tgt.AUX1_CHANGED_ON_DT = src.AUX1_CHANGED_ON_DT,
    tgt.AUX2_CHANGED_ON_DT = src.AUX2_CHANGED_ON_DT,
    tgt.AUX3_CHANGED_ON_DT = src.AUX3_CHANGED_ON_DT,
    tgt.AUX4_CHANGED_ON_DT = src.AUX4_CHANGED_ON_DT,
    tgt.W_UPDATE_DT = current_timestamp(),
    tgt.ETL_PROC_WID = 2625278;

In [ ]:
%sql
-- 13) UPDATE CONTROL TABLE: Upsert W_ETL_LOAD_DATES
MERGE INTO workspace.oracle_edw.w_etl_load_dates AS tgt
USING (
  SELECT
    p.DATASOURCE_NUM_ID AS datasource_num_id,
    'SILOS_SIL_GLACCOUNTDIMENSION' AS package_name,
    'W_GL_ACCOUNT_D' AS target_table_name,
    p.ETL_USAGE_CODE AS etl_usage_code,
    p.ETL_PROC_WID AS etl_proc_wid,
    p.EXECUTION_ID AS load_plan_id,
    current_timestamp() AS etl_load_date,
    CASE WHEN p.IS_INCREMENTAL = 'Y' THEN '1' ELSE '0' END AS committed,
    date_add(current_timestamp(), -p.PRUNE_DAYS) AS wip_load_start_date
  FROM etl_params p
) src
ON tgt.datasource_num_id = src.datasource_num_id
  AND tgt.package_name = src.package_name
  AND tgt.etl_usage_code = src.etl_usage_code
WHEN MATCHED THEN
  UPDATE SET
    tgt.target_table_name = src.target_table_name,
    tgt.etl_proc_wid = src.etl_proc_wid,
    tgt.load_plan_id = src.load_plan_id,
    tgt.wip_load_start_date = src.wip_load_start_date,
    tgt.etl_load_date = src.etl_load_date,
    tgt.committed = src.committed
WHEN NOT MATCHED THEN
  INSERT (
    datasource_num_id, package_name, target_table_name, etl_usage_code,
    etl_proc_wid, load_plan_id, wip_load_start_date, last_max_date, etl_load_date, committed
  )
  VALUES (
    src.datasource_num_id, src.package_name, src.target_table_name, src.etl_usage_code,
    src.etl_proc_wid, src.load_plan_id, src.wip_load_start_date, NULL, src.etl_load_date, src.committed
  );

In [ ]:
%sql
-- 14) INSERT HISTORY LOG: Record ETL execution in history table
INSERT INTO workspace.oracle_edw.w_etl_load_dates_log
(
  DATASOURCE_NUM_ID,
  PACKAGE_NAME,
  TARGET_TABLE_NAME,
  ETL_USAGE_CODE,
  ETL_PROC_WID,
  LOAD_PLAN_ID,
  SESSION_ID,
  WIP_LOAD_START_DATE,
  LAST_MAX_DATE,
  ETL_LOAD_DATE,
  COMMITTED
)
SELECT
  p.DATASOURCE_NUM_ID,
  'SILOS_SIL_GLACCOUNTDIMENSION',
  'W_GL_ACCOUNT_D',
  p.ETL_USAGE_CODE,
  p.ETL_PROC_WID,
  p.EXECUTION_ID,
  p.ETL_PROC_WID AS SESSION_ID,
  date_add(current_timestamp(), -p.PRUNE_DAYS) AS WIP_LOAD_START_DATE,
  NULL AS LAST_MAX_DATE,
  current_timestamp() AS ETL_LOAD_DATE,
  CASE WHEN p.IS_INCREMENTAL = 'Y' THEN '1' ELSE '0' END AS COMMITTED
FROM etl_params p;

In [ ]:
%sql
-- 15) CLEANUP: Drop temporary views
DROP VIEW IF EXISTS stg_account_with_action;
DROP VIEW IF EXISTS stg_account_transformed;
DROP VIEW IF EXISTS domain_map_fin_stmt;
DROP VIEW IF EXISTS domain_map_gl_acct_cat;
DROP VIEW IF EXISTS stg_account_with_lookups;
DROP VIEW IF EXISTS lkp_natural_account;
DROP VIEW IF EXISTS lkp_user_changed;
DROP VIEW IF EXISTS lkp_user_created;
DROP VIEW IF EXISTS etl_params;